In [34]:
import pandas as pd
import numpy as np
import math

In [427]:
class Node:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.is_leaf = True
        self.column = None
        self.split_point = None
        self.children = None
    
    def is_pure(self):
        p = self.probabilities()
        if p[0] == 1 or p[1] == 1:
            return True
        return False

    def split(self, depth=0):
        X, y = self.X, self.y
        if self.is_leaf and not self.is_pure():
           
            splits = [ best_split_point(X, y, column) for column in range(X.shape[1]) ]
            splits.sort()
            gini, split_point, column = splits[0]
            self.is_leaf = False
            self.column = column
            self.split_point = split_point
            
            below = X.loc[:,column] <= split_point
            above = X.loc[:,column] > split_point 
            
            self.children = [
                Node(X[below], y[below]),
                Node(X[above], y[above])
            ]
            
            if depth:
                for child in self.children:
                    child.split(depth-1)
                    
                    
    def probabilities(self):
        return np.array([
        np.mean(self.y == 0),
        np.mean(self.y == 1),
    ])

    def predict_proba(self, row):
        if self.is_leaf:
            return self.probabilities()
        else:
            #print(row,self.column,self.split_point)
            if row[self.column] <= self.split_point:
                return self.children[0].predict_proba(row)
            else:
                return self.children[1].predict_proba(row)
    def formatted(self, indent=0):
        if self.is_leaf:
            s = "Leaf({p[0]:.3f}, {p[1]:.3f})".format(p=self.probabilities())
        else:
            s = "Branch(X{column} <= {split_point})\n{left}\n{right}".format(
                column=self.column, 
                split_point=self.split_point,
                left=self.children[0].formatted(indent+1),
                right=self.children[1].formatted(indent+1))

        return "    " * indent + s

    def __str__(self):
        return self.formatted()

    def __repr__(self):
        return str(self)

In [428]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=3):
        self.max_depth = int(max_depth)
        self.root = None
        
    def fit(self, X, y):
        self.root = Node(X, y)
        self.root.split(self.max_depth)
        
    def predict_proba(self, X):
        results = []
        for index, row in X.iterrows():
            #print(row)
            p = self.root.predict_proba(row)
            results += [p]
        return np.array(results)
            
    def predict(self, X):
        predictions = (self.predict_proba(X)[:, 1] > 0.5)
        #print(predictions)
        return predictions


In [429]:
def best_split_point(X, y, column):
    
    X.reset_index(inplace=True,drop=True)
    y.reset_index(inplace=True,drop=True)
    # sorting y by the values of X makes
    # it almost trivial to count classes for
    # above and below any given candidate split point. 
    ordering = np.argsort(X[column]).values
    
    
    y = y[0].values

    classes = y[ordering]
    
    #print(ordering,classes)

    # these vectors tell us how many of each
    # class are present "below" (to the left)
    # of any given candidate split point. 
    class_0_below = (classes == 0).cumsum()
    class_1_below = (classes == 1).cumsum()
    
    #print(class_0_below,class_1_below)
    
    # Subtracting the cummulative sum from the total
    # gives us the reversed cummulative sum. These
    # are how many of each class are above (to the
    # right) of any given candidate split point.
    #
    # Because class_0_below is a cummulative sum
    # the last value in the array is the total sum.
    # That means we don't need to make another pass
    # through the array just to get the total; we can
    # just grab the last element. 
    class_0_above = class_0_below[-1] - class_0_below
    class_1_above = class_1_below[-1] - class_1_below
    
    # below_total = class_0_below + class_1_below
    below_total = np.arange(1, len(y)+1)
    # above_total = class_0_above + class_1_above
    above_total = np.arange(len(y)-1, -1, -1)

    # we can now calculate Gini impurity in a single
    # vectorized operation. The naive formula would be:
    #
    #     (class_1_below/below_total)*(class_0_below/below_total)
    # 
    # however, divisions are expensive and we can get this down
    # to only one division if we combine the denominator term.
    gini = class_1_below * class_0_below / (below_total ** 2) + \
           class_1_above * class_0_above / (above_total ** 2)

    gini[np.isnan(gini)] = 1
    
    # we need to reverse the above sorting to
    # get the rule into the form C_n < split_value. 
    best_split_rank = np.argmin(gini)
    best_split_gini = gini[best_split_rank]
    best_split_index = np.argwhere(ordering == best_split_rank).item(0)
    #print(np.argwhere(ordering == best_split_rank).item(0),column)

    best_split_value = X.loc[best_split_index,column]
    
    
    return best_split_gini, best_split_value, column

In [430]:
# a small classification data set with 30 to get with. 
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()

X = pd.DataFrame(breast_cancer['data'])
y = pd.DataFrame(breast_cancer['target'])

model = DecisionTreeClassifier(max_depth=4)
model.fit(X, y)
y_hat = model.predict(X)
p_hat = model.predict_proba(X)[:,1]

/home/local/ADMOGLILABS/kulvinder.singh/.local/lib/python3.6/site-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in true_divide


In [431]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
print(confusion_matrix(y, y_hat))
print('Accuracy:', accuracy_score(y, y_hat))


[[193  19]
 [ 18 339]]
Accuracy: 0.9349736379613357
